В моем распоряжении данные о книгах (включая информацию об авторах и издателях), а также отзывы и ревью об этих книгах. Моя основная цель изучить данные и подготовить код для поиска поределенной информации среди этих данных.

####  Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [4]:
# Установка параметров отображения
pd.set_option('display.max_colwidth', None)

####  Знакомство

In [5]:
# формируем запрос и выводим данные
query = '''SELECT * 
FROM books
LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
query = ''' SELECT COUNT(*) 
FROM books
;'''
get_sql_data(query)

,count
0,1000


In [7]:
query = ''' SELECT COUNT(DISTINCT book_id) 
FROM books
;'''
get_sql_data(query)

,count
0,1000


In [8]:
query = '''SELECT MIN(num_pages) AS max_pages
FROM books
;'''
get_sql_data(query)

,max_pages
0,14


Самая маленькая книга содержит 14 страниц, посмотрим что это за книга?

In [9]:
query = ''' SELECT title, num_pages
FROM books
WHERE num_pages = (SELECT MIN(num_pages) AS max_pages FROM books)
;'''
get_sql_data(query)

,title,num_pages
0,Moo Baa La La La!,14
1,Without Fail (Jack Reacher #6),14


Две книги с самым маленьким количеством страниц, первая "Moo Baa La La La!"	это детская литература, а второе "Without Fail" больше похоже на ошибку в данных.

In [10]:
query = '''SELECT MAX(num_pages) AS max_pages
FROM books
;'''
get_sql_data(query)

,max_pages
0,2690


Самая большая книга содержит 2690 страниц, стало интересно, что это за книга?

In [11]:
query = ''' SELECT title, num_pages
FROM books
WHERE num_pages = (SELECT MAX(num_pages) AS max_pages FROM books)
;'''
get_sql_data(query)

,title,num_pages
0,Harry Potter Boxed Set Books 1-5 (Harry Potter #1-5),2690


Оказалось, это не совсем книга, это набор из пяти книг.

In [12]:
query = ''' SELECT MIN(publication_date) AS min_date,
MAX(publication_date) AS max_date
FROM books

;'''
get_sql_data(query)

,min_date,max_date
0,1952-12-01,2020-03-31


В таблице "books" содержится тысяча строк, с данными за период с 1 декабря 1952 года по 31 марта 2020

In [13]:
query = ''' SELECT * 
FROM authors
LIMIT 5
;'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [14]:
query = ''' SELECT COUNT(*) 
FROM authors
;'''
get_sql_data(query)

,count
0,636


In [15]:
query = ''' SELECT COUNT(DISTINCT author) 
FROM authors
;'''
get_sql_data(query)

,count
0,636


В таблице "authors" 636 строк, каждая представляет собой имя автора без повторов.

In [16]:
query = '''SELECT *
FROM publishers
LIMIT 5
;'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [17]:
query = ''' SELECT COUNT(*) 
FROM publishers
;'''
get_sql_data(query)

,count
0,340


In [18]:
query = ''' SELECT COUNT(DISTINCT publisher) 
FROM publishers
;'''
get_sql_data(query)

,count
0,340


В таблице "publisher" содержится 340 строк с названиями издательств без повторов.

In [19]:
query = '''SELECT *
FROM ratings
LIMIT 5
;'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [20]:
query = '''SELECT COUNT(*)
FROM ratings
;'''
get_sql_data(query)

,count
0,6456


In [21]:
query = '''SELECT COUNT(DISTINCT username)
FROM ratings
;'''
get_sql_data(query)

,count
0,160


In [22]:
query = '''SELECT AVG(rating)
FROM ratings
;'''
get_sql_data(query)

,avg
0,3.928284


Таблица "ratings" содержит 6456 строк с отзывами от 160 пользователей. Средняя оценка пользователей 3.9

In [23]:
query = '''SELECT *
FROM reviews
LIMIT 5
;'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


In [24]:
query = '''SELECT COUNT(*)
FROM reviews
;'''
get_sql_data(query)

,count
0,2793


In [25]:
query = '''SELECT COUNT(DISTINCT username)
FROM reviews
;'''
get_sql_data(query)

,count
0,160


Таблица "reviews" содержит 2793 строк с ревью от 160 пользователей.

Итого, у нас данные о 1000 книг от 636 автороы за период с 1 декабря 1952 года по 31 марта 2020. Размер книг от 14 до 2690 страниц. Также у нас есть данные о рейтингах и ревью от 160 пользовтелей, средний рейтинг книг 3.9 балла. 

####  Выполнение заданий

**Задача** Посчитайте, сколько книг вышло после 1 января 2000 года;

In [26]:
query = '''SELECT COUNT(*) AS total_books
FROM books
WHERE publication_date > '2000-01-01' 
;'''
get_sql_data(query)

,total_books
0,819


В базе содержиться информаци о публикации 819 книг с января 2000 года

**Задача** Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [27]:
query = '''SELECT
    b.title,
    COUNT(DISTINCT r.review_id) AS review_count,
    AVG(ra.rating) AS average_rating
FROM
    books b
    LEFT JOIN reviews r ON b.book_id = r.book_id
    LEFT JOIN ratings ra ON b.book_id = ra.book_id
GROUP BY
    b.book_id,
    b.title
ORDER BY
    average_rating DESC
;'''
get_sql_data(query)

,title,review_count,average_rating
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,The Walking Dead Book One (The Walking Dead #1-12),2,5.00
2,Light in August,2,5.00
3,Wherever You Go There You Are: Mindfulness Meditation in Everyday Life,2,5.00
4,Captivating: Unveiling the Mystery of a Woman's Soul,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twenty-first Century,3,2.25
996,His Excellency: George Washington,2,2.00
997,Drowning Ruth,3,2.00
998,Junky,2,2.00


В результате получается таблица, которая содержит название каждой книги, вместе с подсчитанным количеством обзоров, и средней оценкой. Самый высокий рейтинг у  книги "Arrows of the Queen", ровно пять баллов, а самый низкий рейтинг в 1.5 баллов у "Harvesting the Heart". Интересно, что количество обзоров всего два, как для книги с самой низкой оценкой, так и с высокой.

**Задача** Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

Эту задачу можно разбить на этапы, сначала получим количество книг больше 50 страниц

In [28]:
query = '''SELECT
  book_id,
  publisher_id
FROM
  books
WHERE
  num_pages > 50;
; '''
get_sql_data(query)

,book_id,publisher_id
0,1,93
1,2,336
2,3,135
3,4,309
4,5,268
...,...,...
987,996,147
988,997,297
989,998,212
990,999,331


Теперь мы можем дополнить запрос, посчитав книги для каждого издательства

In [29]:
query = '''
SELECT
    publisher_id,
    COUNT(book_id) AS number_of_books
FROM
    books
WHERE
    num_pages > 50
GROUP BY
    publisher_id
ORDER BY
    number_of_books DESC;
; '''
get_sql_data(query)

,publisher_id,number_of_books
0,212,42
1,309,31
2,116,25
3,217,24
4,33,19
...,...,...
329,34,1
330,225,1
331,138,1
332,245,1


Осталось подтянуть название издательства из таблицы с издателями

In [30]:
query = '''SELECT
  p.publisher,
  COUNT(b.book_id) AS number_of_books
FROM
  publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE
  b.num_pages > 50
GROUP BY
  p.publisher
ORDER BY
  number_of_books DESC
LIMIT 1
;'''
get_sql_data(query)

,publisher,number_of_books
0,Penguin Books,42


Согласно нашей базе, издательство Penguin Books выпустило наибольшее число книг толще 50 страниц

**Задача** Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;

Сначала найдем книги с 50 оценками и более

In [31]:
query = '''
SELECT
    book_id,
    COUNT(rating_id) AS rating_count
FROM
    ratings
GROUP BY
    book_id
HAVING
    COUNT(rating_id) >= 50
; '''
get_sql_data(query)

,book_id,rating_count
0,75,84
1,750,88
2,545,66
3,948,160
4,488,61
5,696,59
6,722,74
7,627,57
8,733,56
9,779,62


Возникает вопрос - сколько оценок выставлено каждой книге?

In [32]:
query = '''SELECT
    b.book_id,
    b.title,
    COUNT(r.rating) AS rating_count
FROM
    books b
LEFT JOIN ratings r ON b.book_id = r.book_id
GROUP BY
    b.book_id, b.title
ORDER BY
    rating_count DESC
;'''
get_sql_data(query)

,book_id,title,rating_count
0,948,Twilight (Twilight #1),160
1,750,The Hobbit or There and Back Again,88
2,673,The Catcher in the Rye,86
3,75,Angels & Demons (Robert Langdon #1),84
4,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),82
...,...,...,...
995,465,Naked Empire (Sword of Truth #8),2
996,55,A Woman of Substance (Emma Harte Saga #1),2
997,652,The Body in the Library (Miss Marple #3),2
998,790,The Magicians' Guild (Black Magician Trilogy #1),2


Больше всего оценок у Сумерек. Нам осталось посчитать среднее оценок для каждого автора, и подтянуть его имя из таблицы авторов

In [33]:
query = '''SELECT
  a.author,
  AVG(sub.avg_rating) AS average_rating
FROM (
  SELECT
    b.book_id,
    b.author_id,
    AVG(r.rating) AS avg_rating,
    COUNT(r.rating) AS rating_count
  FROM
    books b
  JOIN ratings r ON b.book_id = r.book_id
  GROUP BY
    b.book_id
  HAVING
    COUNT(r.rating) >= 50
) AS sub
JOIN authors a ON sub.author_id = a.author_id
GROUP BY
  a.author
ORDER BY
  average_rating DESC
LIMIT 1
;'''
get_sql_data(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Высшая оценка у Джоан Роулинг, автора серии Гарри Поттера

**Задача** Посчитайте среднее количество обзоров от пользователей, которые поставилибольше 48 оценок.

Сначала найдем пользователей, которые поставили больше 48 оценок

In [34]:
query = '''SELECT
    username
FROM
    ratings
GROUP BY
    username
HAVING
    COUNT(rating_id) > 48
;'''
get_sql_data(query)

,username
0,walter49
1,sfitzgerald
2,jennifermiller
3,andreaeaton
4,xdavis
5,vcarter
6,paul88
7,lesliegibbs
8,susan85
9,martinadam


Дальше нужно посчитать общее количество обзоров, которые написали эти пользователи.

In [35]:
query = '''SELECT
    COUNT(review_id) AS total_reviews
FROM
    reviews
WHERE
    username IN (SELECT username FROM ratings
        GROUP BY 
            username
        HAVING
            COUNT(rating_id) > 48
  )
;'''
get_sql_data(query)

,total_reviews
0,312


Всего 312 обзоров. Теперь используя это число мы можем посчитать среднее количество обзоров на пользователя

In [36]:
query = '''SELECT
  AVG(review_count) AS average_review_count
FROM (
  SELECT
    username,
    COUNT(review_id) AS review_count
  FROM
    reviews
  WHERE
    username IN (
      SELECT
        username
      FROM
        ratings
      GROUP BY
        username
      HAVING
        COUNT(rating_id) > 48
    )
  GROUP BY
    username
) AS subquery
;'''
get_sql_data(query)

,average_review_count
0,24.0


Получилось, из тех кто поставил больше 48 оценок приходится в среднем 24 обзора на пользователя. 

#### Общий вывод

Мы проанализировали данные о 1000 книгах, написанных 636 авторами, за период с 1 декабря 1952 года по 31 марта 2020. Эти книги имеют разнообразный объем от 14 до 2690 страниц. В нашем распоряжении также информация о рейтингах и отзывах, предоставленных 160 пользователями, со средним рейтингом книг в 3.9 балла.

Мы узнали из базы, что начиная с января 2000 года  было опубликованно 819 книг. Мы составили таблицу, в которой для каждой книги указаны название, количество отзывов и средняя оценка. Книга "Arrows of the Queen" занимает первое место с максимальным рейтингом в пять баллов, в то время как "Harvesting the Heart" имеет самый низкий рейтинг в 1.5 балла, при этом у обеих книг всего по два отзыва, что делает сомнительным информацию об оценке.

По нашим данным, издательство Penguin Books выпустило больше всего книг объемом более 50 страниц по сравнению с другими издательствами - всего 42 штуки за исследуемый период. Наибольшее количество оценок - 160 штук - получила книга "Сумерки". Самая высокая средняя оценка, равная 4.2 балла, у Джоан Роулинг, автора серии книг о Гарри Поттере.

Всего в базе насчитывается 312 отзывов от 160 пользователей. Интересно, что среди тех, кто оставил более 48 оценок, в среднем приходится 24 отзыва на пользователя.